In [2]:
import pandas as pd

In [3]:
tcia_file = "tcia_data_categories.xlsx"
df_tcia = pd.read_excel(tcia_file)
selected_rows_imaging = ["MR", "CT", "PT"]
df_tcia2 = df_tcia[df_tcia["Modality"].isin(selected_rows_imaging)]
print(df_tcia2.head())

  Modality  count  AP-6H6G  AP-6M60  AP-95DK  AP-97PG  AP-9GTR  AP-CL9I  \
1       CT   1140        6        3        7        6        5        4   
4       MR    414        0        0        0        0        0        0   
5       PT     91        1        1        1        0        1        1   

   AP-DRVN  C3L-00189  ...  TCGA-ZF-AA4X  TCGA-ZF-AA51  TCGA-ZF-AA52  \
1        7          3  ...             2             0             2   
4        0          0  ...             0            12             0   
5        0          0  ...             0             0             0   

   TCGA-ZF-AA53  TCGA-ZF-AA54  TCGA-ZF-AA56  TCGA-ZF-AA58  TCGA-ZF-AA5H  \
1             0             2             0             2             3   
4            11             0            19             0             0   
5             0             0             0             0             0   

   TCGA-ZF-AA5N  TCGA-ZF-AA5P  
1             8             4  
4             0             0  
5            

In [4]:
gdc_file = "gdc_data_categories.xlsx"
df_gdc = pd.read_excel(gdc_file)
df_gdc["Modality"] = df_gdc.iloc[:, :4].astype(str).agg("_".join, axis=1)
df_gdc = df_gdc[["Modality"] + [col for col in df_gdc.columns if col != "Modality"]]

selected_rows_omics = [
    #    'Biospecimen_Biospecimen Supplement_nan_BCR Auxiliary XML',
    #    'Biospecimen_Biospecimen Supplement_nan_BCR Biotab',
    #    'Biospecimen_Biospecimen Supplement_nan_BCR SSF XML',
    #    'Biospecimen_Biospecimen Supplement_nan_BCR XML',
    #    'Biospecimen_Slide Image_Diagnostic Slide_SVS',
    #    'Biospecimen_Slide Image_Tissue Slide_SVS',
    #    'Clinical_Clinical Supplement_nan_BCR Biotab',
    #    'Clinical_Clinical Supplement_nan_BCR OMF XML',
    #    'Clinical_Clinical Supplement_nan_BCR XML',
    #    'Clinical_Pathology Report_nan_PDF',
    #    'Copy Number Variation_Allele-specific Copy Number Segment_Genotyping Array_TXT',
    #    'Copy Number Variation_Allele-specific Copy Number Segment_WGS_TXT',
    #    'Copy Number Variation_Copy Number Segment_Genotyping Array_TXT',
    #    'Copy Number Variation_Copy Number Segment_WGS_TXT',
       'Copy Number Variation_Gene Level Copy Number_Genotyping Array_TSV',
    #    'Copy Number Variation_Gene Level Copy Number_WGS_TSV',
    #    'Copy Number Variation_Masked Copy Number Segment_Genotyping Array_TXT',
       'DNA Methylation_Masked Intensities_Methylation Array_IDAT',
    #    'DNA Methylation_Methylation Beta Value_Methylation Array_TXT',
       'Proteome Profiling_Protein Expression Quantification_Reverse Phase Protein Array_TSV',
       'Simple Nucleotide Variation_Masked Somatic Mutation_WXS_MAF',
       'Transcriptome Profiling_Gene Expression Quantification_RNA-Seq_TSV',
    #    'Transcriptome Profiling_Isoform Expression Quantification_miRNA-Seq_TSV',
    #    'Transcriptome Profiling_Isoform Expression Quantification_miRNA-Seq_TXT',
    #    'Transcriptome Profiling_Raw Intensities_Expression Array_CEL',
    #    'Transcriptome Profiling_miRNA Expression Quantification_miRNA-Seq_TSV',
    #    'Transcriptome Profiling_miRNA Expression Quantification_miRNA-Seq_TXT'
       ]
df_gdc2 = df_gdc[df_gdc["Modality"].isin(selected_rows_omics)]
print(df_gdc2.head())

                                             Modality  \
14  Copy Number Variation_Gene Level Copy Number_G...   
17  DNA Methylation_Masked Intensities_Methylation...   
19  Proteome Profiling_Protein Expression Quantifi...   
20  Simple Nucleotide Variation_Masked Somatic Mut...   
21  Transcriptome Profiling_Gene Expression Quanti...   

                  data_category                          data_type  \
14        Copy Number Variation             Gene Level Copy Number   
17              DNA Methylation                 Masked Intensities   
19           Proteome Profiling  Protein Expression Quantification   
20  Simple Nucleotide Variation            Masked Somatic Mutation   
21      Transcriptome Profiling     Gene Expression Quantification   

          experimental_strategy data_format  counts  AP-6H6G  AP-6M60  \
14             Genotyping Array         TSV    1104        0        0   
17            Methylation Array        IDAT    1435        0        0   
19  Reverse Phase

In [5]:
common_cols = df_tcia2.columns.intersection(df_gdc2.columns)
df_combined = pd.concat([df_tcia2[common_cols], df_gdc2[common_cols]], ignore_index=True)

rows_to_sum = df_combined[df_combined["Modality"].isin(selected_rows_imaging)]
totals = (rows_to_sum.sum(numeric_only=True)>0).astype(bool)
totals_row = totals.to_dict()
totals_row['Modality'] = 'Images Available'
df_combined = pd.concat([df_combined, pd.DataFrame([totals_row])], ignore_index=True)

rows_to_sum = df_combined[df_combined["Modality"].isin(selected_rows_omics)]
totals = (rows_to_sum.sum(numeric_only=True)>0).astype(bool)
totals_row = totals.to_dict()
totals_row['Modality'] = 'Omics Available'
df_combined = pd.concat([df_combined, pd.DataFrame([totals_row])], ignore_index=True)

row1 = df_combined[df_combined["Modality"] == "Images Available"].iloc[0]
row2 = df_combined[df_combined["Modality"] == "Omics Available"].iloc[0]
and_row = row1.drop("Modality") & row2.drop("Modality")
and_row["Modality"] = "Both Available"
df_combined = pd.concat([df_combined, pd.DataFrame([and_row])], ignore_index=True)
df_combined


final_n = df_combined[df_combined["Modality"] == 'Both Available'].sum(numeric_only=True)
final_n

AP-6H6G         0
AP-6M60         0
AP-95DK         0
AP-97PG         0
AP-9GTR         0
               ..
TCGA-ZF-AA56    1
TCGA-ZF-AA58    1
TCGA-ZF-AA5H    1
TCGA-ZF-AA5N    1
TCGA-ZF-AA5P    1
Length: 1450, dtype: int64

In [6]:
final_n = (df_combined[df_combined["Modality"] == 'Both Available']).sum(axis=1, numeric_only=True).iloc[0].item()
print(final_n)

row = df_combined[df_combined["Modality"] == 'Both Available'].iloc[0]
patientID_list = row[row == True].index.tolist()
print(patientID_list)

df = pd.DataFrame(patientID_list)
df.to_excel('patientIDs_final.xlsx', index=False, header=False)

1441
['C3L-00189', 'C3L-00263', 'C3L-00401', 'C3L-00422', 'C3L-00598', 'C3L-00599', 'C3L-00608', 'C3L-00610', 'C3L-00622', 'C3L-00625', 'C3L-00766', 'C3L-00770', 'C3L-00771', 'C3L-00792', 'C3L-00799', 'C3L-00800', 'C3L-00812', 'C3L-00815', 'C3L-00817', 'C3L-00893', 'C3L-00938', 'C3L-00947', 'C3L-00961', 'C3L-00963', 'C3L-01000', 'C3L-01031', 'C3L-01032', 'C3L-01034', 'C3L-01036', 'C3L-01064', 'C3L-01246', 'C3L-01247', 'C3L-01248', 'C3L-01277', 'C3L-01330', 'C3L-01352', 'C3L-01458', 'C3L-01459', 'C3L-01462', 'C3L-01463', 'C3L-01464', 'C3L-01465', 'C3L-01467', 'C3L-01687', 'C3L-01689', 'C3L-01703', 'C3L-01732', 'C3L-01744', 'C3L-01924', 'C3L-01925', 'C3L-01954', 'C3L-01958', 'C3L-01960', 'C3L-01962', 'C3L-01976', 'C3L-02109', 'C3L-02112', 'C3L-02115', 'C3L-02116', 'C3L-02118', 'C3L-02202', 'C3L-02399', 'C3L-02403', 'C3L-02412', 'C3L-02508', 'C3L-02513', 'C3L-02515', 'C3L-02606', 'C3L-02610', 'C3L-02613', 'C3L-02744', 'C3L-02747', 'C3L-02841', 'C3L-02888', 'C3L-02890', 'C3L-03123', 'C3L-0